사전순 정렬 대신 nature sort를 사용한다.

In [2]:
# cell 1

import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


먼저 파일을 읽어서 카테고리 이름을 요소로 갖는 string list를 만든다.

In [3]:
# cell 2

import os

path = '/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data'
folder_list = os.listdir(path)

category = []
# 카테고리 string 저장할 list

for i in folder_list:
    if not i.endswith('.DS_Store'):
    	# .DS_Store 빼고 추가
        category.append(i)
        
category.sort(key=natural_keys)

print(category)

['child', 'culture', 'economy', 'education', 'health', 'life', 'person', 'policy', 'society']


각 카테고리 안에 있는 파일 이름을 요소로 갖는 2D string list를 만든다.

In [ ]:
# cell 3

fileName2D = []
# row: categoryIdx, column: 해당 카테고리의 파일 이름

for categoryIdx in range(len(category)):
    path = '/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data/'+category[categoryIdx]
    file_list = os.listdir(path)
    #해당 카테고리 안에 있는 파일명이 담긴 string list

    txt_list = []
    for i in file_list:
        if i.endswith('.txt') and not i.endswith('.DS_Store'):
            #파일 형식이 .txt로 끝나는 파일 이름만 배열에 추가
            txt_list.append(i)

    txt_list.sort(key=natural_keys)
    fileName2D.append(txt_list)

for i in fileName2D:
    print(len(i), end=" ")
    # 각 카테고리마다의 파일 개수 출력
print()
print(fileName2D)


모든 파일의 형태소를 요소로 갖는 중복없는 corpus를 만든다.
row: docNum, 0번 column: category,  나머지 column: corpus 인 2D Dataframe을 만든다

빈 DataFrame을 생성했을 때 행과 열이 둘 다 없으면 데이터를 추가하거나 수정할 수 없다.
열 정보를 갖고 있어야 행 데이터를 추가할 수 있기 때문에 첫 column인 category를 추가해준다.

In [9]:
# cell 4

import pandas as pd

TF = pd.DataFrame(columns=range(1))
TF.columns = ['category']

corpus = []
docNum = 0
#문서 번호
for categoryIdx in range(len(category)):
    for fileIdx in range(len(fileName2D[categoryIdx])):
        file = open("/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data/"+category[categoryIdx]+"/"+fileName2D[categoryIdx][fileIdx], "r")

        lines = file.readlines()
        #한 줄씩 string list로 저장
        #row에 doc+docNum 저장
        #카테고리 열 값에 categoryIdx 저장
        for line in lines:
            morp = line.split('\t')[-1].split('+')[0].strip('\n')
            #'\t'를 기준으로 뒤, '+'를 기준으로 앞의 string을 잘라 배열에 저장
            
            if morp != '' and morp not in corpus:
                #자른 string이 빈 문자열이 아니고 corpus안에서 중복되지 않을 때 요소 추가
                corpus.append(morp)
                
corpus.sort(key=natural_keys)
#ㄱㄴㄷ 순 정렬
print(corpus)

,category


형태소의 출현 횟수를 구하고 내림차순 정렬 후, corpus의 크기가 5000을 초과할 경우에는 5000번 부터의 요소를 삭제한다.